# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17245838903938719104
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11332668621
locality {
  bus_id: 1
}
incarnation: 13904563505339572670
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'a': 3, 'sentence': 21, 'is': 19, 'by': 10, 'this': 18, 'over': 7, 'study': 13, 'quick': 2, 'lazy': 8, 'fox': 5, 'jumps': 6, 'lexicography': 15, 'brown': 4, 'short': 20, 'won': 16, 'my': 12, 'dog': 9, 'jove': 11, 'prize': 17, 'the': 1, 'of': 14}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max(len(sentence) for sentence in x)
    return np.array([np.concatenate([sentence, np.zeros(length - len(sentence))]) for sentence in x])

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [ 1.  2.  4.  5.  6.  7.  1.  8.  9.  0.]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [ 10.  11.  12.   2.  13.  14.  15.  16.   3.  17.]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [ 18.  19.   3.  20.  21.   0.   0.   0.   0.   0.]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index) + 1
french_vocab_size = len(french_tokenizer.word_index) + 1

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 200
French vocabulary size: 345


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '_'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

def tokens_to_text(tokens, tokenizer):
    """
    Turn tokens being fed into neural network into text using the tokenizer
    (redundant with original sentences, but nice to have this in the same format as logits_to_text)
    :param tokens: Tokens for words being fed into a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the tokens
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '_'
    return ' '.join([index_to_words[token] for token in list(tokens)])

print('`tokens_to_text` function loaded.')

`logits_to_text` function loaded.
`tokens_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [11]:
# Setting up a one-hot encoder, since otherwise we're implicitly assigning semantic closeness to 
# words with nearby indices

def one_hot_encode(token_sentences, vocab_size):
    one_hot_encoded = []
    sentence_length = token_sentences.shape[1]
    for token_sentence in token_sentences:
        new_sentence = np.zeros(shape=(sentence_length, vocab_size))
        for i, token in enumerate(token_sentence):
            try:
                new_sentence[i, int(token[0])] = 1.0
            except:
                new_sentence[i, int(token)] = 1.0
        one_hot_encoded.append(np.array(new_sentence))
    return np.array(one_hot_encoded)


In [15]:
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.utils import to_categorical

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    learning_rate = 0.001
    GRU_units = 512
    
    # TODO: Build the layers
    model = Sequential()
    model.add(GRU(input_shape=input_shape[1:],
                  units=GRU_units,
                  return_sequences=True))
    model.add(TimeDistributed(Dense(units=french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(tmp_x.shape)

one_hot_tmp_x = one_hot_encode(tmp_x, english_vocab_size)

print(one_hot_tmp_x.shape)

# Train the neural network
simple_rnn_model = simple_model(
    one_hot_tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(one_hot_tmp_x, preproc_french_sentences, batch_size=256, epochs=10, validation_split=0.2)


(137861, 21, 1)
(137861, 21, 200)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 34s - loss: 1.5059 - acc: 0.6839 - val_loss: 0.5759 - val_acc: 0.8348
Epoch 2/10
110288/110288 [==============================] - 34s - loss: 0.4264 - acc: 0.8673 - val_loss: 0.3407 - val_acc: 0.8899
Epoch 3/10
110288/110288 [==============================] - 34s - loss: 0.3019 - acc: 0.9011 - val_loss: 0.2689 - val_acc: 0.9120
Epoch 4/10
110288/110288 [==============================] - 34s - loss: 0.2542 - acc: 0.9149 - val_loss: 0.2413 - val_acc: 0.9186
Epoch 5/10
110288/110288 [==============================] - 34s - loss: 0.2278 - acc: 0.9224 - val_loss: 0.2259 - val_acc: 0.9228
Epoch 6/10
110288/110288 [==============================] - 34s - loss: 0.2104 - acc: 0.9271 - val_loss: 0.2160 - val_acc: 0.9251
Epoch 7/10
110288/110288 [==============================] - 34s - loss: 0.1988 - acc: 0.9301 - val_loss: 0.2039 - val_acc: 0.9294
Epoch

In [16]:
# Print prediction(s) (One at a time slows things, but that's ok.)
for i in range(20):
    print("English sentence:")
    print(logits_to_text(one_hot_tmp_x[i], english_tokenizer))
    pred_logits = simple_rnn_model.predict(one_hot_tmp_x[i:i + 1])[0]
    print("French translation:")
    print(logits_to_text(pred_logits, french_tokenizer))
    print("-----------------")

English sentence:
new jersey is sometimes quiet during autumn and it is snowy in april _ _ _ _ _ _ _ _
French translation:
new jersey est parfois calme en l' automne et il est neigeux en avril _ _ _ _ _ _ _
-----------------
English sentence:
the united states is usually chilly during july and it is usually freezing in november _ _ _ _ _ _
French translation:
les états unis est généralement froid en juillet et il est généralement en printemps _ _ _ _ _ _ _
-----------------
English sentence:
california is usually quiet during march and it is usually hot in june _ _ _ _ _ _ _ _
French translation:
californie est généralement calme en mois et il est généralement chaud en juin _ _ _ _ _ _ _ _
-----------------
English sentence:
the united states is sometimes mild during june and it is cold in september _ _ _ _ _ _ _
French translation:
les états unis est parfois doux en juin et il est froid en septembre _ _ _ _ _ _ _
-----------------
English sentence:
your least liked fruit is the grape 

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [20]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.001
    english_embedding_size = 64
    GRU_units = 512
    
    # TODO: Build the layers
    model = Sequential()
    
    # Embedding english input
    model.add(Embedding(english_vocab_size, english_embedding_size, input_length=input_shape[1]))
    
    # RNN layer
    model.add(GRU(units=GRU_units, return_sequences=True))
    
    # Dense layer
    
    model.add(TimeDistributed(Dense(units=french_vocab_size, activation='softmax')))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# Reshaping the input to work with the embedding RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))
print(tmp_x.shape)

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=10, validation_split=0.2)


(137861, 21)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 31s - loss: 1.9081 - acc: 0.5911 - val_loss: 0.9206 - val_acc: 0.7669
Epoch 2/10
110288/110288 [==============================] - 30s - loss: 0.5716 - acc: 0.8415 - val_loss: 0.4020 - val_acc: 0.8756
Epoch 3/10
110288/110288 [==============================] - 30s - loss: 0.3309 - acc: 0.8951 - val_loss: 0.2908 - val_acc: 0.9062
Epoch 4/10
110288/110288 [==============================] - 30s - loss: 0.2656 - acc: 0.9119 - val_loss: 0.2450 - val_acc: 0.9184
Epoch 5/10
110288/110288 [==============================] - 30s - loss: 0.2302 - acc: 0.9221 - val_loss: 0.2327 - val_acc: 0.9221
Epoch 6/10
110288/110288 [==============================] - 30s - loss: 0.2100 - acc: 0.9276 - val_loss: 0.2114 - val_acc: 0.9276
Epoch 7/10
110288/110288 [==============================] - 30s - loss: 0.1967 - acc: 0.9314 - val_loss: 0.2016 - val_acc: 0.9304
Epoch 8/10
110288/110288 [

In [18]:
# Print prediction(s) (One at a time slows things, but that's ok.)
for i in range(20):
    print("English sentence:")
    print(tokens_to_text(tmp_x[i], english_tokenizer))
    pred_logits = embed_rnn_model.predict(tmp_x[i:i + 1])[0]
    print("French translation:")
    print(logits_to_text(pred_logits, french_tokenizer))
    print("-----------------")

English sentence:
new jersey is sometimes quiet during autumn and it is snowy in april _ _ _ _ _ _ _ _
French translation:
new jersey est parfois calme en l' automne et il neigeux neigeux en avril _ _ _ _ _ _ _
-----------------
English sentence:
the united states is usually chilly during july and it is usually freezing in november _ _ _ _ _ _
French translation:
les états unis est généralement froid en juillet et il est généralement en mai _ _ _ _ _ _ _
-----------------
English sentence:
california is usually quiet during march and it is usually hot in june _ _ _ _ _ _ _ _
French translation:
californie est généralement calme en mars et il est généralement chaud en juin _ _ _ _ _ _ _ _
-----------------
English sentence:
the united states is sometimes mild during june and it is cold in september _ _ _ _ _ _ _
French translation:
les états unis est parfois doux en juin et il est froid en septembre _ _ _ _ _ _ _
-----------------
English sentence:
your least liked fruit is the grape bu

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [21]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate = 0.001
    GRU_units = 512
    
    # TODO: Build the layers
    model = Sequential()
    
    # Bidirectional GRU
    model.add(Bidirectional(GRU(input_shape=input_shape[1:],
                                units=GRU_units,
                                return_sequences=True),
                            merge_mode='ave',
                            input_shape=input_shape[1:]))
    
    # Dense layer
    model.add(TimeDistributed(Dense(units=french_vocab_size, activation='softmax')))
    
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


tests.test_bd_model(bd_model)

# Use same one-hot encoded input we usd for the simple model...

print(one_hot_tmp_x.shape)

# Train the neural network
bd_rnn_model = bd_model(
    one_hot_tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_rnn_model.fit(one_hot_tmp_x, preproc_french_sentences, batch_size=256, epochs=10, validation_split=0.2)


(137861, 21, 200)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 63s - loss: 1.5120 - acc: 0.6613 - val_loss: 0.7480 - val_acc: 0.7884
Epoch 2/10
110288/110288 [==============================] - 62s - loss: 0.5229 - acc: 0.8510 - val_loss: 0.3622 - val_acc: 0.8937
Epoch 3/10
110288/110288 [==============================] - 62s - loss: 0.2801 - acc: 0.9160 - val_loss: 0.2318 - val_acc: 0.9283
Epoch 4/10
110288/110288 [==============================] - 62s - loss: 0.1904 - acc: 0.9407 - val_loss: 0.1701 - val_acc: 0.9481
Epoch 5/10
110288/110288 [==============================] - 62s - loss: 0.1472 - acc: 0.9538 - val_loss: 0.1362 - val_acc: 0.9576
Epoch 6/10
110288/110288 [==============================] - 62s - loss: 0.1189 - acc: 0.9627 - val_loss: 0.1173 - val_acc: 0.9642
Epoch 7/10
110288/110288 [==============================] - 62s - loss: 0.1012 - acc: 0.9683 - val_loss: 0.1059 - val_acc: 0.9673
Epoch 8/10
110288/110

In [23]:
# Print prediction(s) (One at a time slows things, but that's ok.)
for i in range(20):
    print("English sentence:")
    print(logits_to_text(one_hot_tmp_x[i], english_tokenizer))
    pred_logits = bd_rnn_model.predict(one_hot_tmp_x[i:i + 1])[0]
    print("French translation:")
    print(logits_to_text(pred_logits, french_tokenizer))
    print("-----------------")

English sentence:
new jersey is sometimes quiet during autumn and it is snowy in april _ _ _ _ _ _ _ _
French translation:
new jersey est parfois calme au l' automne et il est neigeux en avril _ _ _ _ _ _ _
-----------------
English sentence:
the united states is usually chilly during july and it is usually freezing in november _ _ _ _ _ _
French translation:
les états unis est généralement froid en juillet et il gèle habituellement en novembre _ _ _ _ _ _ _
-----------------
English sentence:
california is usually quiet during march and it is usually hot in june _ _ _ _ _ _ _ _
French translation:
california est généralement calme en mars et il est généralement chaud en juin _ _ _ _ _ _ _ _
-----------------
English sentence:
the united states is sometimes mild during june and it is cold in september _ _ _ _ _ _ _
French translation:
les états unis est parfois légère en juin et il fait froid en septembre _ _ _ _ _ _ _
-----------------
English sentence:
your least liked fruit is the g

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [30]:
from keras.layers import LSTM

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    
    
    learning_rate = 0.001
    encoder_GRU_units = 512
    decoder_GRU_units = 512
    
    model = Sequential()

    # Encoder
    
    # add a simpleRNN layer
    model.add(Bidirectional(GRU(units=encoder_GRU_units), input_shape=input_shape[1:]))
    
    model.add(RepeatVector(output_sequence_length))
    
    # Decoder

    model.add(Bidirectional(GRU(units=decoder_GRU_units, return_sequences=True)))
    
    # Dense layer
    model.add(TimeDistributed(Dense(units=french_vocab_size, activation='softmax')))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

In [41]:
def start_prepender(sentences):
    """
    Takes a list of sentences and returns the list with <START> prepended.
    Seemed to improve accuracy by a couple percent.
    """
    new_sentences = []
    for s in sentences:
        new_sentences.append('<START> ' + s)
    return new_sentences
        

ed_english_sentences = start_prepender(english_sentences)
ed_french_sentences = start_prepender(french_sentences)

ed_preproc_english_sentences, ed_preproc_french_sentences, ed_english_tokenizer, ed_french_tokenizer =\
    preprocess(ed_english_sentences, ed_french_sentences)
    
ed_max_english_sequence_length = ed_preproc_english_sentences.shape[1]
ed_max_french_sequence_length = ed_preproc_french_sentences.shape[1]
ed_english_vocab_size = len(ed_english_tokenizer.word_index) + 1
ed_french_vocab_size = len(ed_french_tokenizer.word_index) + 1


# One-hot encode English input
ed_one_hot_input = one_hot_encode(ed_preproc_english_sentences, ed_english_vocab_size)
print("Shape of one-hot encoded input:", ed_one_hot_input.shape)


Shape of one-hot encoded input: (137861, 16, 201)


In [42]:
# Train the neural network (this is where an error comes up)

ed_model = encdec_model(ed_one_hot_input.shape,
                        ed_max_french_sequence_length,
                        ed_english_vocab_size,
                        ed_french_vocab_size)

ed_model.fit(ed_one_hot_input,
             ed_preproc_french_sentences,
             batch_size=256,
             epochs=10,
             validation_split=0.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 129s - loss: 1.3894 - acc: 0.6525 - val_loss: 0.7905 - val_acc: 0.7677
Epoch 2/10
110288/110288 [==============================] - 128s - loss: 0.5585 - acc: 0.8323 - val_loss: 0.3880 - val_acc: 0.8818
Epoch 3/10
110288/110288 [==============================] - 128s - loss: 0.2692 - acc: 0.9174 - val_loss: 0.2121 - val_acc: 0.9353
Epoch 4/10
110288/110288 [==============================] - 128s - loss: 0.1791 - acc: 0.9437 - val_loss: 0.1461 - val_acc: 0.9535
Epoch 5/10
110288/110288 [==============================] - 128s - loss: 0.1327 - acc: 0.9573 - val_loss: 0.1289 - val_acc: 0.9588
Epoch 6/10
110288/110288 [==============================] - 128s - loss: 0.1076 - acc: 0.9655 - val_loss: 0.1061 - val_acc: 0.9660
Epoch 7/10
110288/110288 [==============================] - 128s - loss: 0.0902 - acc: 0.9708 - val_loss: 0.0945 - val_acc: 0.9703
Epoch 8/10
110288/110288 [======

In [43]:
# Print prediction(s) (One at a time slows things, but that's ok.)
for i in range(20):
    print("English sentence:")
    print(logits_to_text(ed_one_hot_input[i][1:], ed_english_tokenizer))
    pred_logits = ed_model.predict(ed_one_hot_input[i:i + 1])[0]
    print("French translation:")
    print(logits_to_text(pred_logits[1:], ed_french_tokenizer))
    print("-----------------")

English sentence:
new jersey is sometimes quiet during autumn and it is snowy in april _ _
French translation:
new jersey est parfois calme pendant l' automne et il est neigeux en avril _ _ _ _ _ _ _
-----------------
English sentence:
the united states is usually chilly during july and it is usually freezing in november
French translation:
les états unis est généralement froid en juillet et il gèle habituellement en novembre _ _ _ _ _ _ _
-----------------
English sentence:
california is usually quiet during march and it is usually hot in june _ _
French translation:
california est généralement calme en mars et il est généralement chaud en juin _ _ _ _ _ _ _ _
-----------------
English sentence:
the united states is sometimes mild during june and it is cold in september _
French translation:
les états unis est parfois doux en juin et il fait froid en septembre _ _ _ _ _ _ _
-----------------
English sentence:
your least liked fruit is the grape but my least liked is the apple _
French

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [45]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.001
    english_embedding_size = 64
    encoder_GRU_units = 512
    decoder_GRU_units = 512
    
    model = Sequential()
    
    # Embedding
    
    # Embedding english input
    model.add(Embedding(english_vocab_size, english_embedding_size, input_length=input_shape[1]))
    
    # Encoder
    
    # add a simpleRNN layer
    model.add(Bidirectional(GRU(units=encoder_GRU_units), input_shape=input_shape[1:]))
    
    model.add(RepeatVector(output_sequence_length))
    
    # Decoder

    model.add(Bidirectional(GRU(units=decoder_GRU_units, return_sequences=True)))
    
    # Dense layer
    model.add(TimeDistributed(Dense(units=french_vocab_size, activation='softmax')))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')

# TODO: Train the final model
final_rnn_model = model_final(
    ed_preproc_english_sentences.shape,
    ed_max_french_sequence_length,
    ed_english_vocab_size,
    ed_french_vocab_size)
final_rnn_model.fit(ed_preproc_english_sentences, ed_preproc_french_sentences, batch_size=256, epochs=20, validation_split=0.2)

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 127s - loss: 1.6239 - acc: 0.6037 - val_loss: 0.9668 - val_acc: 0.7179
Epoch 2/20
110288/110288 [==============================] - 125s - loss: 0.7624 - acc: 0.7699 - val_loss: 0.6182 - val_acc: 0.8086
Epoch 3/20
110288/110288 [==============================] - 125s - loss: 0.4859 - acc: 0.8484 - val_loss: 0.3904 - val_acc: 0.8752
Epoch 4/20
110288/110288 [==============================] - 125s - loss: 0.3008 - acc: 0.9051 - val_loss: 0.2380 - val_acc: 0.9244
Epoch 5/20
110288/110288 [==============================] - 125s - loss: 0.2025 - acc: 0.9349 - val_loss: 0.1882 - val_acc: 0.9384
Epoch 6/20
110288/110288 [==============================] - 125s - loss: 0.1635 - acc: 0.9461 - val_loss: 0.1465 - val_acc: 0.9508
Epoch 7/20
110288/110288 [==============================] - 125s - loss: 0.1319 - acc: 0.9560 - val_loss: 0.1345 - val_acc: 0.9546
Epoch 8/20
11

In [48]:
# Print prediction(s) (One at a time slows things, but that's ok.)
for i in range(20):
    print("English sentence:")
    print(tokens_to_text(ed_preproc_english_sentences[i][1:], ed_english_tokenizer))
    pred_logits = final_rnn_model.predict(ed_preproc_english_sentences[i:i + 1])[0]
    print("French translation:")
    print(logits_to_text(pred_logits[1:], ed_french_tokenizer))
    print("-----------------")

English sentence:
new jersey is sometimes quiet during autumn and it is snowy in april _ _
French translation:
new jersey est parfois calme pendant l' automne et il est neigeux en avril _ _ _ _ _ _ _
-----------------
English sentence:
the united states is usually chilly during july and it is usually freezing in november
French translation:
les états unis est généralement froid en juillet et il gèle habituellement en novembre _ _ _ _ _ _ _
-----------------
English sentence:
california is usually quiet during march and it is usually hot in june _ _
French translation:
california est généralement calme en mars et il est généralement chaud en juin _ _ _ _ _ _ _ _
-----------------
English sentence:
the united states is sometimes mild during june and it is cold in september _
French translation:
les états unis est parfois légère en juin et il fait froid en septembre _ _ _ _ _ _ _
-----------------
English sentence:
your least liked fruit is the grape but my least liked is the apple _
Fren

## Prediction (IMPLEMENTATION)

In [51]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    # (I'm just using the already trained model)
    model = final_rnn_model

    
    # I'm gonna edit below the line to avoid displaying <START> when printing.
    # I marked the three lines I modified with comments.
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]][1:]))  # Added [1:]
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]][1:]))  # Added [1:]
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]][1:]))               # Added [1:]


final_predictions(ed_preproc_english_sentences, ed_preproc_french_sentences, ed_english_tokenizer, ed_french_tokenizer)

Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?